<a href="https://colab.research.google.com/github/PyMainCoder/Muser/blob/main/Muser_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Muser_AI by Glitch_Verse ver 3.0

In [ ]:
!nvidia-smi

In [ ]:
#@title Clone the inference repo
!git clone https://github.com/hmartiro/riffusion-inference
%cd riffusion-inference

In [ ]:
#@title Install requirements (you may need to restart the kernel after this)
!pip install -r requirements.txt
!pip install gradio
!pip install --upgrade pillow

In [ ]:
#@title Imports
from diffusers import DiffusionPipeline
from riffusion.spectrogram_image_converter import SpectrogramImageConverter
from riffusion.spectrogram_params import SpectrogramParams
from io import BytesIO
from IPython.display import Audio

pipe = DiffusionPipeline.from_pretrained("riffusion/riffusion-model-v1")
pipe = pipe.to("cuda")

In [ ]:
#@title Define a `predict` function

params = SpectrogramParams()
converter = SpectrogramImageConverter(params)

def predict(prompt, negative_prompt):
    spec = pipe(
        prompt,
        negative_prompt=negative_prompt,
        width=768,
    ).images[0]

    wav = converter.audio_from_spectrogram_image(image=spec)
    wav.export('output.wav', format='wav')
    return 'output.wav', spec

In [ ]:
#@title Run with Colab interface
prompt = "solo piano piece, classical"#@param {type:"string"}
negative_prompt = "drums"#@param {type:"string"}

path, spec = predict(prompt, negative_prompt)

display(spec)
Audio('output.wav')

In [ ]:
#@title Run a Gradio demo
import gradio as gr

gr.Interface(
    predict,
    inputs=["text", "text"],
    outputs=[gr.outputs.Audio(type='filepath'), gr.outputs.Image(type='pil')],
    title="Riffusion",
).launch(share=True, debug=True)

In [ ]:
#@title Upload your own files for style transfer
#@markdown #### Drop your audio files (.wav is best) in Colab's file uploader, then type the filename below and run

from google.colab import files
# uploaded = files.upload()

from scipy.io import wavfile
import numpy as np
from PIL import Image

filename = "rondo_alla_turca.wav"#@param {type:"string"}

# read uploaded file to wav
rate, data = wavfile.read(f'../{filename}')

# resample from 48000 to 44100
# from scipy.signal import resample
# data = resample(data, int(data.shape[0] * 44100 / 48000))

# convert to mono
data = np.mean(data, axis=1)

# convert to float32
data = data.astype(np.float32)

# take a random 7 second slice of the audio
data = data[rate*7:rate*14]

spectrogram = spectrogram_from_waveform(
    waveform=data,
    sample_rate=rate,
    # width=768,
    n_fft=8192,
    hop_length=512,
    win_length=8192,
)

def image_from_spectrogram(
    spectrogram: np.ndarray, max_volume: float = 50, power_for_image: float = 0.25
) -> Image.Image:
    """
    Compute a spectrogram image from a spectrogram magnitude array.
    """
    # Apply the power curve
    data = np.power(spectrogram, power_for_image)

    # Rescale to 0-255
    data = data * 255 / max_volume

    # Invert
    data = 255 - data

    # Convert to a PIL image
    image = Image.fromarray(data.astype(np.uint8))

    # Flip Y
    image = image.transpose(Image.FLIP_TOP_BOTTOM)

    # Convert to RGB
    image = image.convert("RGB")

    return image

spec = image_from_spectrogram(spectrogram)
# img.save('../rondo_alla_turca.png')
# display(img)

# Audio(data, rate=rate)

# wav2 = wav_bytes_from_spectrogram_image(img)
# with open("../inverse-spectro.wav", "wb") as f:
#     f.write(wav2[0].getbuffer())

# Audio('../inverse-spectro.wav')

In [ ]:
#@title Audio-to-audio based on the previous generated sound: define new pipeline

import torch
from diffusers import StableDiffusionImg2ImgPipeline

# load the pipeline
device = "cuda"
MODEL_ID = "riffusion/riffusion-model-v1"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(MODEL_ID, torch_dtype=torch.float16)

pipe = pipe.to(device)

In [ ]:
#@title Audio-to-audio: run inference
prompt = "epic orchestra symphony" #@param {type:"string"}

images = pipe(
    prompt=prompt,
    image=spec,
    strength=0.5,
    guidance_scale=7
).images

wav = converter.audio_from_spectrogram_image(images[0])
wav.export('audio2audio.wav', format='wav')
Audio('audio2audio.wav')